In [2]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch import nn
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import json

# Assurez-vous que toutes les fonctions et classes nécessaires sont définies ici ou importées correctement

# Fonction pour charger un fichier JSONL
def load_jsonl(file_path):
    with open(file_path, 'r') as file:
        data = [json.loads(line) for line in file]
    return pd.DataFrame(data)

# Définir le modèle et autres configurations ici
class TextEncoder(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim):
        super(TextEncoder, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, hidden_dim)

    def forward(self, x):
        x = self.embedding(x)
        x = torch.relu(self.fc(x))
        return x

class ComplexModel(nn.Module):
    def __init__(self, vocab_size, num_numeric_features, embed_dim, hidden_dim, output_dim):
        super(ComplexModel, self).__init__()
        self.text_encoder = TextEncoder(vocab_size, embed_dim, hidden_dim)
        self.numeric_fc = nn.Linear(num_numeric_features, hidden_dim)
        self.fc1 = nn.Linear(hidden_dim * 2, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, numeric_data, text_data):
        text_features = self.text_encoder(text_data)
        numeric_features = torch.relu(self.numeric_fc(numeric_data))
        combined_features = torch.cat((text_features, numeric_features), dim=1)
        x = torch.relu(self.fc1(combined_features))
        x = self.fc2(x)
        return x

# Définir le jeu de données personnalisé
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, num_data, text_data, labels):
        self.num_data = torch.tensor(num_data, dtype=torch.float)
        self.text_data = text_data
        self.labels = torch.tensor(labels.values, dtype=torch.float)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.num_data[idx], self.text_data[idx], self.labels[idx]

# Fonction d'évaluation
def evaluate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    all_labels = []
    all_preds = []
    with torch.no_grad():
        for num_data, text_data, labels in dataloader:
            num_data, text_data, labels = num_data.to(device), text_data.to(device), labels.to(device)
            outputs = model(num_data, text_data)
            outputs = outputs.squeeze()  
            labels = labels.squeeze()  
            loss = criterion(outputs, labels)
            running_loss += loss.item() * num_data.size(0)
            all_labels.append(labels)
            all_preds.append(outputs)
    
    epoch_loss = running_loss / len(dataloader.dataset)
    all_labels = torch.cat(all_labels)
    all_preds = torch.cat(all_preds)
    rmse = mean_squared_error(all_labels.cpu().numpy(), all_preds.cpu().numpy(), squared=False)
    r2 = r2_score(all_labels.cpu().numpy(), all_preds.cpu().numpy())
    mae = mean_absolute_error(all_labels.cpu().numpy(), all_preds.cpu().numpy())
    return epoch_loss, rmse, r2, mae

# Configurer le dispositif (CPU ou GPU)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Paramètres du modèle
vocab_size = 10000  # Remplacez par la taille de votre vocabulaire
num_numeric_features = 3  # Remplacez par le nombre de caractéristiques numériques
embed_dim = 50  # Dimension de l'embedding
hidden_dim = 128  # Dimension cachée
output_dim = 1  # Dimension de la sortie

# Initialisation du modèle
model = ComplexModel(vocab_size, num_numeric_features, embed_dim, hidden_dim, output_dim)
model.to(device)

# Charger le modèle sauvegardé
best_model_path = '/home/carolus/Documents/school/green_ia/best_models/174.ci'  # Assurez-vous que ce chemin est correct
model.load_state_dict(torch.load(best_model_path, map_location=device))  # Utiliser map_location pour charger sur CPU ou GPU approprié
model.eval()
print('Meilleur modèle chargé pour les prédictions.')

# Charger le fichier JSONL de validation
validation_df = load_jsonl('/home/carolus/Documents/school/green_ia/data/00_valid_01.jsonl')

# Préparation des données de validation
num_cols = ['groups', 'countries', 'labels_note']
text_cols = ['packaging', 'name', 'ingredients', 'categories']

X_valid = validation_df.drop(columns=['ecoscore_score'])
y_valid = validation_df['ecoscore_score']

X_valid_num = X_valid[num_cols]
X_valid_text = X_valid[text_cols].astype(str)

# Normalisation des données numériques
scaler = StandardScaler()
X_valid_num = scaler.fit_transform(X_valid_num)  # Assurez-vous d'utiliser les mêmes statistiques que pour l'entraînement

# Préparer les données textuelles
tokenizer = get_tokenizer('basic_english')
MAX_SEQ_LEN = 50  # Paramètre pour la longueur maximale des séquences

# Créer le vocabulaire
def build_vocab(texts):
    vocab = build_vocab_from_iterator(map(tokenizer, texts))
    vocab.insert_token('<unk>', 0)
    vocab.insert_token('<pad>', 1)
    vocab.set_default_index(vocab['<unk>'])
    return vocab

all_texts = pd.concat([X_valid_text[col] for col in text_cols])
vocab = build_vocab(all_texts)

# Transformer les textes en indices
def text_to_indices(text):
    tokens = tokenizer(text)
    indices = [vocab[token] for token in tokens]
    if len(indices) < MAX_SEQ_LEN:
        indices += [vocab['<pad>']] * (MAX_SEQ_LEN - len(indices))
    else:
        indices = indices[:MAX_SEQ_LEN]
    return torch.tensor(indices)

def text_data_to_tensor(text_data):
    return torch.stack([text_to_indices(text) for text in text_data])

X_valid_text_combined = X_valid_text.apply(lambda x: ' '.join(x), axis=1)
X_valid_text_indices = text_data_to_tensor(X_valid_text_combined)

# Créer le DataLoader pour les données de validation
valid_dataset = CustomDataset(X_valid_num, X_valid_text_indices, y_valid)
valid_loader = DataLoader(valid_dataset, batch_size=64, shuffle=False)

# Critère de perte
criterion = nn.MSELoss()

# Évaluer le modèle sur les données de validation
valid_loss, valid_rmse, valid_r2, valid_mae = evaluate(model, valid_loader, criterion, device)

print(f'Validation loss: {valid_loss:.4f}, Validation RMSE: {valid_rmse:.4f}, Validation R2: {valid_r2:.4f}, Validation MAE: {valid_mae:.4f}')


RuntimeError: Error(s) in loading state_dict for ComplexModel:
	size mismatch for text_encoder.embedding.weight: copying a param with shape torch.Size([745823, 50]) from checkpoint, the shape in current model is torch.Size([10000, 50]).